# Pulse Sequence Design Using Reinforcement Learning

Implementing deep deterministic policy gradient (DDPG) to learn pulse sequence design for spin systems. The [OpenAI SpinningUp resource](https://spinningup.openai.com/en/latest/algorithms/ddpg.html#pseudocode) has a good theoretical background on DDPG which I used to implement the algorithm below.

DDPG is designed for _continuous_ action spaces, which is the ultimate goal for this project (to apply pulses with arbitrary axes of rotation, rotation angles, and times, instead of limiting to pi/2 pulses along X or Y). However, that means the algorithm is less suited to constrained versions of the problem, such as only applying pi/2 pulses of a certain length about X or Y.

For training, the following reward function was used
$$
r = -\log\left( 1- \left| \text{Tr}\left( \frac{U_\text{target}^\dagger U_\text{exp}}{2^N} \right) \right| \right)
= -\log\left( 1- \text{fidelity}(U_\text{target}^\dagger, U_\text{exp}) \right)
$$
For example, if the fidelity is $0.999$, then the reward $r = -\log(0.001) = 3$. 

<!-- For the policy function, I need to perform gradient ascent with the following gradient
$$
\nabla_\theta 1/|B| \sum_{s \in B} Q_\phi (s, \pi_\theta(s))
$$

And for the Q-function, perform gradient descent with
$$
\nabla_\phi 1/|B| \sum_{(s,a,r,s',d) \in B} (Q_\phi(s,a) - y(r,s',d))^2
$$ -->

Other resources:

- https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough#define_the_loss_and_gradient_function
- https://www.tensorflow.org/guide/migrate#customize_the_training_step

In [ ]:
import spinSimulation as ss
import rlPulse as rlp
import numpy as np
import scipy.linalg as spla
import importlib
from tqdm import tqdm

In [ ]:
importlib.reload(ss)
importlib.reload(rlp)

# Initialize spin system

This sets the parameters of the system ($N$ spin-1/2 particles, which corresponds to a Hilbert space with dimension $2^N$). For the purposes of simulation, $\hbar \equiv 1$.

The total internal Hamiltonian is given by
$$
H_\text{int} = C H_\text{dip} + \Delta \sum_i^N I_z^{(i)}
$$
where $C$ is the coupling strength, $\Delta$ is the chemical shift strength (each spin is assumed to be identical), and $H_\text{dip}$ is given by
$$
H_\text{dip} = \sum_{i,j}^N d_{i,j} \left(3I_z^{(i)}I_z^{(j)} - \mathbf{I}^{(i)} \cdot \mathbf{I}^{(j)}\right)
$$

The WAHUHA pulse sequence is designed to remove the dipolar interaction term from the internal Hamiltonian. The pulse sequence is $\tau, P_{-x}, \tau, P_{y}, \tau, \tau, P_{-y}, \tau, P_{x}, \tau$.
The zeroth-order average Hamiltonian for the WAHUHA pulse sequence is
$$
H_\text{WHH} = \Delta / 3 \sum_i^N I_x^{(i)} + I_y^{(i)} + I_z^{(i)}
$$

In [ ]:
N = 4
dim = 2**N

# pulse = .25e-6    # duration of pulse
# delay = 3e-6      # duration of delay
coupling = 5e3    # coupling strength
delta = 500       # chemical shift strength (for identical spins)

(x,y,z) = (ss.x, ss.y, ss.z)
(X,Y,Z) = ss.getTotalSpin(N, dim)

Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
HWHH0 = ss.getHWHH0(X,Y,Z,delta)

# Initialize RL algorithm

An "action" performed on the system corresponds to an RF-pulse applied to the system. A pulse can be parametrized by the axis of rotation (e.g. $(\theta, \phi)$, but for now $\theta = \pi/2$ is assumed so the axis of rotation lies in the xy-plane), the rotation angle, and the duration of the pulse.

The state of the system can correspond to the propagator, but because the propagator grows exponentially (it has $4^N$ elements for an $N$-spin system) and the pulse sequence determines the propagator, the state is represented by the pulse sequence instead.

The target network parameters $\theta_\text{target}$ are updated by
$$
\theta_\text{target} = \rho \theta_\text{target} + (1-\rho)\theta
$$

TODO figure out if this buffer size makes sense

In [ ]:
sDim = 3 # state represented by sequences of actions...
aDim = 3 # action = [phi, rot, time]

numExp = 2000 # how many experiences to "play" through and learn from
bufferSize = 500 # size of the replay buffer (i.e. how many experiences to keep in memory).
batchSize = 50 # size of batch (subset of replay buffer) to use as training for actor and critic.
p = 1 # action noise parameter
polyak = 0.75 # polyak averaging parameter
gamma = 0.5 # future reward discount rate

printEvery = 50
updateAfter = 500 # start updating actor/critic networks after this many episodes
updateEvery = 50  # update networks every __ episodes
numUpdates = 2 # how many training updates to perform on a random subset of experiences (s,a,r,s1,d)
randomizeDipolarEvery = 10
lowerNoiseAfter = 500

# calculate beta, the weight to adjust fidelity values at different times
# beta = 1/10e-6 * np.log(1+1e-6)

pDiff = (0-p)/(numExp-lowerNoiseAfter)

Initialize the actor and critic, as well as target actor and target critic. The actor learns the policy function
$$
\pi_\theta: S \to A, s \mapsto a
$$
that picks the optimal action $a$ for a given state $s$, with some set of parameters $\theta$ (in this case weights/biases in the neural network). The critic learns the Q-function
$$
Q_\phi: S \times A \to \mathbf{R}, (s,a) \mapsto q
$$
where $q$ is the total expected rewards by doing action $a$ on a state $s$, and $\phi$ is the parameter set for the Q-function model. The target actor/critic have different parameter sets $\theta_\text{target}$ and $\phi_\text{target}$.

The "environment" keeps track of the system state, and calculates rewards after each episode.

The replay buffer keeps track of the most recent episodes.

In [ ]:
actor = rlp.Actor(sDim,aDim,None)
actorTarget = rlp.Actor(sDim,aDim,None)
critic = rlp.Critic(sDim,aDim,None, gamma)
criticTarget = rlp.Critic(sDim,aDim,None, gamma)
env = rlp.Environment(N, dim, sDim, HWHH0, X, Y)

actorTarget.setParams(actor.getParams())
criticTarget.setParams(critic.getParams())

replayBuffer = rlp.ReplayBuffer(bufferSize)

## DDPG algorithm

In [ ]:
rMat = np.zeros((numExp,))
aMat = np.zeros((numExp,aDim))
timeMat = np.zeros((numExp, 2)) # record length of episode so far and number of pulses
# keep track of when resets/updates happen
resetStateEps = []
updateEps = []

for i in tqdm(range(numExp)):
    # randomize dipolar coupling strengths for Hint
    if i > 0 and i % randomizeDipolarEvery == 0:
        Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
    
    s = env.getState()
    # get action based on current state and some level of noise
    a = rlp.clipAction(actor.predict(env.state) + rlp.actionNoise(p))
    # evolve state based on action
    env.evolve(a, Hint)
    # get reward
    r = env.reward()
    
    # record episode data
    aMat[i,:] = a
    rMat[i] = r
    timeMat[i,:] = [env.t, np.sum(env.state[:,2] != 0)]
    
    # get updated state, and whether it's a terminal state
    s1 = env.getState()
    d = env.isDone()
    replayBuffer.add(s,a,r,s1,d)
    
    # update noise parameter
    if i > lowerNoiseAfter:
        p += pDiff
    
    # CHECK IF TERMINAL
    if d:
        env.reset()
        resetStateEps.append(i)
    # UPDATE NETWORKS
    if (i > updateAfter) and (i % updateEvery == 0):
#         print("updating actor/critic networks (episode {})".format(i))
        updateEps.append(i)
        for update in range(numUpdates):
            batch = replayBuffer.getSampleBatch(batchSize)
            # train critic
            critic.trainStep(batch, actorTarget, criticTarget)
            # train actor
            actor.trainStep(batch, critic)
            # update target networks
            criticTarget.updateParams(critic, polyak)
            actorTarget.updateParams(actor, polyak)
            

# Results

See how the rewards and actions change over time as the actor/critic (hopefully) learn how to control the spin system.

Looking at the histogram below, most rewards are very small (which makes sense, considering a large subset of rewards are from random actions to begin with). There is a slight bump of rewards centered around 3 (corresponding to fidelities of around 0.999).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(rMat, bins=20, color='black', label='rewards')
plt.title('Rewards histogram')
plt.legend()

The rewards increase as the algorithm learns (which is a good thing!), but seem to plateau near 4 once the algorithm converges.

In [ ]:
%matplotlib inline

plt.plot(rMat, 'ok', label='rewards')
ymin, ymax = plt.ylim()
plt.vlines(updateEps, ymin, ymax, color='red', alpha=0.2, label='updates')
#plt.vlines(resetStateEps, ymin, ymax, color='blue', alpha=0.2, linestyles='dashed', label='state reset')
plt.title('Rewards for each episode')
plt.xlabel('Episode number')
plt.ylabel('Reward')
plt.legend()

Below are the actions performed on the system. Random noise is added to actions for early episodes (to explore the action space), and gradually less noise is added as the algorithm converges.

In [ ]:
%matplotlib inline

plt.plot(aMat[:,0], 'ok', label='phi')
plt.title('Phi action')
ymin, ymax = plt.ylim()
plt.vlines(updateEps, ymin, ymax, color='red', alpha=0.2, label='updates')
#plt.vlines(resetStateEps, ymin, ymax, color='blue', alpha=0.2, linestyles='dashed', label='state reset')
plt.xlabel('Episode number')
plt.ylabel('Phi action')
plt.legend()

In [ ]:
%matplotlib inline

plt.plot(aMat[:,1], 'ok', label='rot')
plt.title('Rot action')
ymin, ymax = plt.ylim()
plt.vlines(updateEps, ymin, ymax, color='red', alpha=0.2, label='updates')
#plt.vlines(resetStateEps, ymin, ymax, color='blue', alpha=0.2, linestyles='dashed', label='state reset')
plt.xlabel('Episode number')
plt.ylabel('Rot action')
plt.legend()

In [ ]:
%matplotlib inline

plt.plot(aMat[:,2], 'ok', label='time')
plt.title('Time action')
ymin, ymax = plt.ylim()
plt.vlines(updateEps, ymin, ymax, color='red', alpha=0.2, label='updates')
#plt.vlines(resetStateEps, ymin, ymax, color='blue', alpha=0.2, linestyles='dashed', label='state reset')
plt.xlabel('Episode number')
plt.ylabel('Time action')
plt.legend()

In [ ]:
%matplotlib inline

plt.plot(timeMat[:,0], 'ok', label='time')
plt.title('Pulse sequence length (time)')
ymin, ymax = plt.ylim()
plt.vlines(updateEps, ymin, ymax, color='red', alpha=0.2, label='updates')
#plt.vlines(resetStateEps, ymin, ymax, color='blue', alpha=0.2, linestyles='dashed', label='state reset')
plt.xlabel('Episode number')
plt.ylabel('Pulse sequence length (s)')
plt.legend()

In [ ]:
%matplotlib inline

plt.plot(timeMat[:,1], 'ok', label='time')
plt.title('Pulse sequence length (number of pulses)')
ymin, ymax = plt.ylim()
plt.vlines(updateEps, ymin, ymax, color='red', alpha=0.2, label='updates')
#plt.vlines(resetStateEps, ymin, ymax, color='blue', alpha=0.2, linestyles='dashed', label='state reset')
plt.xlabel('Episode number')
plt.ylabel('Number of pulses')
plt.legend()

Below prints the pulse sequences with the highest rewards in the replay buffer. It seems like the algorithm is only learning to apply a single pulse that rotates $2\pi$ about the x-axis. This pulse "sequence" has reasonably high fidelity ($0.99998$), but is clearly not the WAHUHA sequence.

In [ ]:
rBuffer = np.array([_[2] for _  in replayBuffer.buffer])
indices = rBuffer.argsort()
for i in range(1,5):
#     print(replayBuffer.buffer[indices[-i]])
    rlp.printAction(replayBuffer.buffer[indices[-i]][3])
    print("reward: ", round(rBuffer[indices[-i]],3))

In [ ]:
# evaluate average fidelity for a pulse sequence

fidelities = np.zeros((50,))
for i in range(50):
    Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
    Uexp = rlp.actionToPropagator(N, dim, np.array([0, 1, .5]), Hint, X, Y)
    Utarget = ss.getPropagator(HWHH0, (7.11+3.47+10)*1e-6)
    fidelities[i] = ss.fidelity(np.power(Utarget,1), np.power(Uexp,1))
print("mean fidelity: ", np.mean(fidelities), "std dev: ", np.std(fidelities), "max: ", np.max(fidelities))